In [1]:
import rooms.dataset
import render_optimized as render
import torch
import torch.nn as nn
import metrics
import train
import os
import numpy as np
import evaluate
import trace1

In [2]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#training parameters

n_fibonacci = 128 #128 
late_stage_model= "UniformResidual" #"UniformResidual"
toa_perturb = True #True
model_transmission = False #False

skip_train = False #False
continue_train = False #False

n_epochs = 500 #1000 
batch_size = 7 #4 #4 nel test ##############If dataset has different microphone types better if batch_size divides N_train and the number of microphones of different type!!!!!!
lr = 1e-2 #1e-2
pink_noise_supervision = True #True
pink_start_epoch = 250 #500
fs = 48000 #48000 

load_dir= 'precomputed/' + 'dampenedBase' #+ dataset_name

skip_inference = False #False
skip_music = True #False
skip_eval = False #False
skip_binaural = True #False

valid = False #False #Evaluate on valid instead of test

In [4]:
# Omnidirectional case

loss_fcn = metrics.training_loss_with_decay

In [ ]:
dataset_name = "dampenedBase"

for i in range (5):

    D = rooms.dataset.dataLoader(dataset_name)

    R = render.Renderer(n_surfaces=len(D.all_surfaces), n_fibonacci=n_fibonacci,
                        late_stage_model=late_stage_model,
                        toa_perturb = toa_perturb, model_transmission=model_transmission).to(device)
    
    # Use multiple GPUs if available
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        R = nn.DataParallel(R).module

    save_dir= '~/dampenedBase_decay50/'+ str(i)

    gt_audio = torch.Tensor(D.RIRs[:, :R.RIR_length])

    def initialize(indices, source_xyz, listener_xyzs, surfaces, load_dir,
                ######################################################
                    rendering_methods,
                    mic_orientations,
                    mic_0_gains,
                    mic_180_loss,
                    cardioid_exponents):
        Ls = []

        for idx in indices:
            L= render.get_listener(source_xyz=source_xyz, listener_xyz = listener_xyzs[idx], surfaces = surfaces, 
                                load_dir = load_dir, load_num = idx, speed_of_sound = D.speed_of_sound, 
                                max_order = D.max_order, parallel_surface_pairs = D.parallel_surface_pairs, 
                                max_axial_order = D.max_axial_order, 
                                ####################################################
                                rendering_method = rendering_methods[idx], mic_orientation = mic_orientations[idx], mic_0_gains = mic_0_gains[idx], mic_180_loss = mic_180_loss[idx], cardioid_exponents = cardioid_exponents[idx])
            Ls.append(L)
        return Ls
    
    """
    Training
    """
    if not skip_train:
        print("Training")

        #Initialize Listeners
        Ls = initialize(indices=D.train_indices,
                        listener_xyzs=D.xyzs,
                        source_xyz=D.speaker_xyz,
                        surfaces=D.all_surfaces,
                        load_dir=load_dir,
                        #######################################
                        rendering_methods = D.rendering_methods, #############should define the rendering method for every listener!!!!!!!!!!!!!!!!!!!!!!!!
                        mic_orientations = D.mic_orientations,#############and all this other stuff!!!!!!!!!!!
                        mic_0_gains= D.mic_0_gains, ############################it's needed for the microphone responses!!!!!!!!!!!!!!!!!!!!!!
                        mic_180_loss = D.mic_180_loss,
                        cardioid_exponents = D.cardioid_exponents)
        if continue_train:
            R.load_state_dict(torch.load(os.path.join(save_dir,"weights.pt"))['model_state_dict'])

        losses = train.train_loop(R=R, Ls=Ls, train_gt_audio=gt_audio[D.train_indices], D=D,
                            n_epochs = n_epochs, batch_size = batch_size, lr = lr, loss_fcn = loss_fcn,
                            save_dir=save_dir,
                            pink_noise_supervision = pink_noise_supervision,
                            pink_start_epoch=pink_start_epoch,
                            continue_train = continue_train, fs=fs)

    else:
        R.load_state_dict(torch.load(os.path.join(save_dir,"weights.pt"))['model_state_dict'])
        R.train = False
        R.toa_perturb = False


    print("test on: ", D.test_indices)

    R = render.Renderer(n_surfaces=len(D.all_surfaces), n_fibonacci=n_fibonacci,
                        late_stage_model=late_stage_model,
                        toa_perturb = toa_perturb, model_transmission=model_transmission).to(device) 
    
    gt_audio = torch.Tensor(D.RIRs[:, :R.RIR_length])

    pt_file = torch.load(save_dir + '/weights.pt', map_location=device)
    R.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
    R.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
    R.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
    R.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
    R.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
    R.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

    R.bp_ord_cut_freqs = nn.Parameter(pt_file['model_state_dict']['bp_ord_cut_freqs'])

    #Note - this function relies on precomputed reflection paths
    #########above in the original code, I added the case where there are no precomputed paths
    def inference(R, source_xyz, xyzs, load_dir, source_axis_1=None, source_axis_2=None):
        """
        Render monoaural RIRs at given precomputed reflection paths.

        Parameters
        ----------
        R: Renderer
            renderer to perform inference on
        source_xyz: np.array (3,)
            3D location of source in meters
        xyzs: np.array (N, 3)
            set of listener locations to render at
        load_dir: str
            directory to load precomputed listener paths
        source_axis_1: np.array (3,)
            first axis specifying virtual source rotation,
            default is None which is (1,0,0)
        source_axis_2: np.array (3,)
            second axis specifying virtual source rotation,
            default is None which is (0,1,0)    

        Returns
        -------
        predictions: np.array (N, T) of predicted RIRs    
        """

        predictions = np.zeros((xyzs.shape[0], R.RIR_length))

        with torch.no_grad():
            R.toa_perturb = False
            for idx in range(xyzs.shape[0]):
                #print(idx, flush=True)
                
                R.mic_direction = D.mic_orientations[idx]
                R.mic_0_gain = D.mic_0_gains[idx]
                R.mic_180_loss = D.mic_180_loss[idx]

                if load_dir is None: 
                    # Tracing from Scratch
                    reflections, transmissions, delays, start_directions, end_directions = (
                        trace1.get_reflections_transmissions_and_delays(
                        source=source_xyz, dest=xyzs[idx], surfaces=D.all_surfaces, speed_of_sound=D.speed_of_sound,
                        max_order=D.max_order,parallel_surface_pairs=D.parallel_surface_pairs, max_axial_order=D.max_axial_order)
                    )
                    
                else:   
                    reflections = np.load(os.path.join(load_dir, "reflections/"+str(idx)+".npy"), allow_pickle=True)
                    transmissions = np.load(os.path.join(load_dir, "transmissions/"+str(idx)+".npy"), allow_pickle=True)
                    delays = np.load(os.path.join(load_dir, "delays/"+str(idx)+".npy"),allow_pickle=True)
                    start_directions = np.load(os.path.join(load_dir, "starts/"+str(idx)+".npy"))
                    end_directions = np.load(os.path.join(load_dir, "ends/"+str(idx)+".npy"))

                L = render.ListenerLocation(
                    source_xyz=source_xyz,
                    listener_xyz=xyzs[idx],
                    n_surfaces=R.n_surfaces,
                    reflections=reflections,
                    transmissions=transmissions,
                    delays=delays,
                    start_directions = start_directions,
                    end_directions=end_directions)
                ########################################################
                #predict = R.render_RIR(L, source_axis_1=source_axis_1, source_axis_2=source_axis_2)
                ########################################################
                predict = R.render_RIR_omni(L, source_axis_1=source_axis_1, source_axis_2=source_axis_2)
                
                predictions[idx] = predict.detach().cpu().numpy()

        return predictions
    
    """
    Inference, rendering RIR
    """
    R.train = False
    R.toa_perturb = False

    pred_dir = os.path.join(save_dir, "predictions")
    if not skip_inference:
        pred_rirs = inference(R=R, source_xyz=D.speaker_xyz, 
                            xyzs=D.xyzs[:(len(D.train_indices)+len(D.test_indices) + len(D.valid_indices))],
                            load_dir=load_dir,
                            source_axis_1= np.array([0.648, -0.762, 0]),####s3 SOURCE ROTATION AXIS
                            source_axis_2= np.array([0.762, 0.648, 0]))
                                                            
        train.makedir_if_needed(pred_dir)
        np.save(os.path.join(pred_dir, "pred_rirs.npy"), pred_rirs)

        if not skip_music:
            pred_music = evaluate.render_music(pred_rirs, D.music_dls)    
            np.save(os.path.join(pred_dir,"pred_musics.npy"), pred_music)
    else:
        pred_rirs = np.load(os.path.join(pred_dir, "pred_rirs.npy"))
        pred_music = np.load(os.path.join(pred_dir, "pred_musics.npy"))
        
        
    """
    Evaluation of Monoaural Audio Using Metrics
    """
    if not skip_eval:
        errors_dir = os.path.join(save_dir, "errors")
        train.makedir_if_needed(errors_dir)
        list_of_metrics = metrics.baseline_metrics

        print("tests", D.test_indices)
        if valid:
            eval_indices = D.valid_indices
        else:
            eval_indices = D.test_indices

        # Evaluating RIR Interp
        for eval_metric in list_of_metrics:

            metric_name = eval_metric.__name__
            errors = evaluate.compute_error(pred_rirs, gt_audio, metric=eval_metric, device="cpu") #original code#changed cuda to cpu!!!!!!!!
            #errors = evaluate.compute_error(pred_rirs[eval_indices], gt_audio[eval_indices], metric=eval_metric)
            
            np.save(os.path.join(errors_dir, "errors_" + metric_name +".npy"), errors)
            print(metric_name + " Metric:", flush=True)
            print(np.mean(errors[eval_indices]))
            #print(np.mean(errors))
            
        # Evaluating Music Interp
        if not skip_music:
            for eval_metric in list_of_metrics:

                metric_name = eval_metric.__name__

                # Computing Error
                errors_music = evaluate.eval_music(pred_music, D.music, eval_metric)
                #errors_music = evaluate.eval_music(pred_music[eval_indices], D.music[eval_indices], eval_metric)
                
                np.save(os.path.join(errors_dir, "errors_music_" + metric_name +".npy"), errors_music)
                print(metric_name + " Music Metric:", flush=True)
                print(np.mean(errors_music[eval_indices]))
                #print(np.mean(errors_music))

Training
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Listener Loading From: precomputed/dampenedBase
Loss:	<function training_loss_with_decay at 0x0000014420E754E0>
Late Network Style	UniformResidual
energy_vector
source_response
directivity_sphere
decay
RIR_residual
spline_values
bp_ord_cut_freqs
training initialized


C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\functional.py:665: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ..\aten\src\ATen\native\SpectralOps.cpp:878.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


test on:  [2, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 18, 19, 20, 22, 24, 25, 26, 28, 29, 30, 32, 33, 34, 36, 37, 38, 40, 41, 42, 44, 45, 48, 49, 50, 52, 53, 54, 56, 57, 58, 60, 61, 62, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 82, 83, 84, 86, 87, 88, 90, 91, 92, 94, 95, 96, 98, 99, 100, 102, 103, 106, 107, 108, 110, 111, 112, 114, 116, 117, 118, 120, 121, 122, 124, 125, 126, 128, 129, 130, 132, 133, 134, 136, 137, 140, 141, 142, 144, 145, 146, 148, 149, 150, 152, 153, 154, 156, 157, 158, 160, 162, 163, 164, 166, 167, 168, 170, 171, 172, 174, 175, 176, 178, 179, 180, 182, 183, 186, 187, 188, 190, 191, 192, 194, 195, 196, 198, 199, 200, 202, 203, 204, 206, 208, 209, 210, 212, 213, 214, 216, 217, 218, 220, 221, 222, 224, 225, 226, 228, 229, 232, 233, 234, 236, 237, 238, 240, 241, 242, 244, 245, 246, 248, 249, 250, 252, 254, 255, 256, 258, 259, 260, 262, 263, 264, 266, 267, 268, 270, 271, 272, 274, 275]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


c:\Users\giova\Documents\Units\IIT\hearinganythinganywhere\evaluate.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  errors[i] = metric(torch.tensor(predicted_audios[i]).to(device), torch.tensor(gt_audios[i]).to(device))


multiscale_log_l1 Metric:
0.20814307803002347
env_loss Metric:
0.07060454449424121
rt60_error Metric:
180.73784209404027
rt60_diff Metric:
0.004758101823913743


MemoryError: Unable to allocate 1.38 GiB for an array with shape (185439984,) and data type float64